## Load and join tables

In [ ]:
import pandas as pd

PROFILES = "/Users/shanglinghsu/7cups-analysis/profiles.csv"
CHAT_RESPONSES = "/Users/shanglinghsu/7cups-analysis/Chat Questionnaire (Responses) - Form Responses 1.tsv"
EXP_RESPONSES = "/Users/shanglinghsu/7cups-analysis/Experiment Questionnaire (Responses) - Form Responses 1.tsv"
PARTICIPANTS = "/Users/shanglinghsu/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/7 Cups Participants.xlsx"
LOG_PATH = "/Users/shanglinghsu/backup_logs/flask_outputs_20221202"

participants = pd.read_excel(PARTICIPANTS)
participants = participants[participants["Username"].notna()]  # Filter mock interivews
participants = participants[participants["Category Chosen"].notna()]  # Filter no-show and no responses
participants["user_id"] = participants["user_id"].astype(pd.Int16Dtype(), copy=False)
participants.reset_index(inplace=True)
participants["Participant Index"] = participants.index + 1
NUM_PARTICIPANTS = len(participants)

profiles = pd.read_csv(PROFILES, header=0, index_col=0)
exp_responses = pd.read_csv(EXP_RESPONSES, header=0, sep="\t")
chat_responses = pd.read_csv(CHAT_RESPONSES, header=0, sep="\t")
chats = pd.read_csv(LOG_PATH + ".csv", header=0)

### Join user tables

In [ ]:
# Can't join() due to poor support of join() on string dtype
def left_join_in_place(left, right, left_on, right_on, rprefix):
    for column in right.columns:
        if column == right_on: continue
        mask, values = [], []
        for i, x in enumerate(left[left_on]):
            mask = right[right_on] == x
            if any(mask):
                values.append(right[column][mask].iloc[0])
            elif rprefix + column in left.columns:
                values.append(left[rprefix + column].loc[i])
            else:
                values.append(pd.NA)
        left[rprefix + column] = values
    return left

EXP_PREFIX = "EXP_"
left_join_in_place(participants, profiles, "Username", "Username", "")
left_join_in_place(participants, exp_responses, "user_id", "What is the 3-digit User ID for this experiment?", EXP_PREFIX)

# Alias for readability
participants["Category Confidence"] = participants[[col for col in participants.columns if "confident" in col][0]]

participants.to_csv("participants_table.csv")

### Join chat tables
To get from `participants`:
- participant user_id
- category
- order
- with tool or not

In [ ]:
participants_chat_df = participants[[col for col in participants.columns if ("id_" in col) or ("ategory" in col) or ("first" in col) or ("user_id" in col)]]

CAT_COL = "Choose Anxiety"
participants_chat_df[CAT_COL] = participants_chat_df["Category Chosen"].apply(lambda x: 1 if x == "A" else 0)

left_join_in_place(chats, participants_chat_df, "chat_id", "chat_id_1", "")
left_join_in_place(chats, participants_chat_df, "chat_id", "chat_id_2", "")
ORDER_COL = "Order"
chats[ORDER_COL] = [0] * len(chats)
chats.loc[chats["chat_id_2"].isna(), ORDER_COL] = 2
chats.loc[chats["chat_id_1"].isna(), ORDER_COL] = 1

SHOW_COL = "Show Suggestions"
chats[SHOW_COL] = [0] * len(chats)
chats.loc[(chats[ORDER_COL] == 1) & (chats["show_suggestion_first"] > 0.5), SHOW_COL] = 1
chats.loc[(chats[ORDER_COL] == 2) & (chats["show_suggestion_first"] < 0.5), SHOW_COL] = 1
chats.head(5)

chats = chats[list(filter(lambda x: ("_1" not in x) and ("_2" not in x) and (EXP_PREFIX not in x) and ("show_suggestion_first" not in x), chats.columns))]
left_join_in_place(chats, chat_responses, "chat_id", "What is the 3-letter Chat ID for this chat?", "")
chats = chats.sort_values(by=['datetime'])

chats = chats[list(filter(lambda x: ((not any(chats[x].isna())) or x.startswith("Any")), chats.columns))]

chats.to_csv("chats_table.csv")

## Convert to numeric and binary values

In [ ]:
from datetime import datetime
BIN_PREFIX = "BIN_"
NUM_PREFIX = "NUM_"

def add_convert_column(df, prefix, field, callback):
    df[prefix + field] = df[field].apply(callback)

### Control variables

In [ ]:
add_convert_column(participants, BIN_PREFIX, "Counseling Background", lambda x: 1 if x else 0)
add_convert_column(participants, BIN_PREFIX, 'show_suggestion_first', lambda x: 1 if x else 0)
participants["Counseling Background"][participants["Counseling Background"].isna()] = 0.5

### Profiles

In [ ]:
add_convert_column(participants, NUM_PREFIX, "Listener Rating", lambda x: float(x.split()[0]))
add_convert_column(participants, NUM_PREFIX, "Progress Path", lambda x: int(x.split()[1].replace(",", "")))
add_convert_column(participants, "OBJ_", "Listener Since", lambda x: datetime.strptime(x, "%b %d, %Y"))
add_convert_column(participants, NUM_PREFIX, "Listener Since", lambda x: (datetime.strptime(x, "%b %d, %Y") - datetime.now()).days)

NUM_COL_PROFILES = [
    "NUM_Listener Rating", 
    "NUM_Progress Path", 
    "NUM_Listener Since",
    "Number of Ratings:",
    "Number of Reviews:",
    "Cheers",
    "People Helped",
    "Chats",
    "Group Support Chats",
    "Listener Group Chats",
    "Forum Posts",
    "Forum Upvotes",
]
participants["NUM_Listener Since"].describe()

### Exp Responses

In [ ]:
FREQ_MAP_NUM = {
    "Never": 0,
    "Very Rarely": 1,
    "Rarely": 2,
    "Occasionally": 3,
    "Frequently": 4,
    "Very Frequently": 5,
}
AGREE_MAP_NUM = {
    "Strongly Disagree": 0,
    "Disagree": 1,
    "Undecided": 2,
    "Agree": 3,
    "Strongly Agree": 4,
}
PROB_MAP_NUM = {
    "Very Probably Not": 0,
    "Probably Not": 1,
    "Possibly": 2,
    "Probably": 3,
    "Very Probably": 4,
    "Definitely": 5,
}
###############################################################################
FREQ_MAP_BIN = {
    "Never": 0,
    "Very Rarely": 0,
    "Rarely": 0,
    "Occasionally": 1,
    "Frequently": 1,
    "Very Frequently": 1,
}
AGREE_MAP_BIN = {
    "Strongly Disagree": 0.0,
    "Disagree": 0.0,
    "Undecided": 0.5,
    "Agree": 1.0,
    "Strongly Agree": 1.0,
}
PROB_MAP_BIN = {
    "Very Probably Not": 0,
    "Probably Not": 0,
    "Possibly": 0,
    "Probably": 1,
    "Very Probably": 1,
    "Definitely": 1,
}
###############################################################################
FREQ_OPTIONS = [
    "Never",
    "Very Rarely",
    "Rarely",
    "Occasionally",
    "Frequently",
    "Very Frequently",
]
AGREE_OPTIONS = [
    "Strongly Disagree",
    "Disagree",
    "Undecided",
    "Agree",
    "Strongly Agree",
]
PROB_OPTIONS = [
    "Very Probably Not",
    "Probably Not",
    "Possibly",
    "Probably",
    "Very Probably",
    "Definitely",
]

In [ ]:
harmful_cols = [col for col in participants.columns if col.startswith(EXP_PREFIX) and "harmful" in col]
use_cols = [col for col in participants.columns if col.startswith(EXP_PREFIX) and "make use of" in col]
freq_cols = [col for col in participants.columns if col.startswith(EXP_PREFIX) and ("often" in col and col not in harmful_cols + use_cols)]
agree_cols = [col for col in participants.columns if col.startswith(EXP_PREFIX) and "As a whole" in col]
prob_cols = [col for col in participants.columns if col.startswith(EXP_PREFIX) and "will you" in col]
checkbox_cols = [col for col in participants.columns if col.startswith(EXP_PREFIX) and ("What do you" in col or "more when" in col)]
demo_cols = [col for col in participants.columns if col.startswith(EXP_PREFIX) and ("age" in col or "preferred pronoun(s)" in col or "device" in col)]
open_cols = [col for col in participants.columns if col.startswith(EXP_PREFIX) and (col == 'EXP_Any other comments?' or "like about" in col or "helps more" in col)]
confident_cols = [col for col in participants.columns if "Confidence" in col]

# unused coloumns
used_cols = harmful_cols + use_cols + freq_cols + agree_cols + prob_cols + checkbox_cols + demo_cols + open_cols + confident_cols
[col for col in participants.columns if col.startswith(EXP_PREFIX) and col not in used_cols]

In [ ]:
for column in freq_cols + use_cols:
    add_convert_column(participants, NUM_PREFIX, column, lambda x: FREQ_MAP_NUM[x])
    add_convert_column(participants, BIN_PREFIX, column, lambda x: FREQ_MAP_BIN[x])
for column in agree_cols:
    add_convert_column(participants, NUM_PREFIX, column, lambda x: AGREE_MAP_NUM[x])
    add_convert_column(participants, BIN_PREFIX, column, lambda x: AGREE_MAP_BIN[x])
for column in prob_cols:
    add_convert_column(participants, NUM_PREFIX, column, lambda x: PROB_MAP_NUM[x])
    add_convert_column(participants, BIN_PREFIX, column, lambda x: PROB_MAP_BIN[x])

## Demographic

In [ ]:
participants['EXP_What is your age?'].describe()[["min", "max", "mean", "std"]]

In [ ]:
from collections import Counter
Counter(participants['Gender'])

## Experiment Questionnaire

### Likert scales

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def reformat_question(question, title):
    question = question.replace(" [-]", "").replace(EXP_PREFIX, "").replace("...", "").replace("[", "").replace("]", "")
    question = question.replace("suggested strategies (blue hints)", "strategies")
    question = question.replace("example responses (white buttons)", "examples")
    question = question.replace(title.replace("...", ""), "")
    return question

def plot_responses(cols, options, title):
    cols = cols[::-1]
    count_df = pd.DataFrame(data={}, columns=options)
    for option in options:
        count_df[option] = (participants[cols] == option).sum()

    count_df.plot(
        kind='barh', 
        stacked=True, 
        cmap="RdYlGn",
        width=0.6,
        figsize=(5, 0.8 * len(cols) + 0.2)
    )
    plt.yticks(range(0,len(count_df.index)), list(map(lambda x: reformat_question(x, title).replace("?", ""), count_df.index))) # not working dunno why
    plt.xlabel('Response Counts')
    plt.title(title)
    plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plot_responses(freq_cols, FREQ_OPTIONS, "How often did the...")
plot_responses(harmful_cols, FREQ_OPTIONS[::-1], 'How often did the...')
plot_responses(agree_cols, AGREE_OPTIONS, "As a whole, was the tool...")
plot_responses(use_cols, FREQ_OPTIONS, "If the tool is provided to you, how often do you think you will make use of it?")
plot_responses(prob_cols, PROB_OPTIONS, "If you can switch the tool on and off as you like, will you switch it on?")

### Checkboxes

In [ ]:
from collections import Counter
from matplotlib.pyplot import figure

def get_default_option_count(col):
    flattened_selections = [b for a in participants[col].apply(lambda x: x.split(", ")) for b in a]
    flattened_selections = list(map(lambda x: "None of the above" if "none" in x.lower() else x, flattened_selections))
    counts = list(Counter(flattened_selections).items())
    counts.sort(key=lambda x: x[1])
    counts = list(filter(lambda x: x[0] == "None of the above" or x[1] > 1, counts))
    count_df = pd.DataFrame(data=counts, columns=["Selection", "Count"])
    return count_df

def plot_checkboxes(col, color):
    count_df = get_default_option_count(col)
    fig, ax = plt.subplots()
    ax.barh(
        width=count_df["Count"],
        y=count_df["Selection"],
        color=color,
    )
    for i, x in enumerate(count_df["Count"]):
        plt.text(x, i, " {} ({:.0f}%)".format(x, x / NUM_PARTICIPANTS * 100))

    plt.xlim(0, 15)
    plt.xlabel('Response Counts')
    plt.title(reformat_question(col, ""))

plot_checkboxes("EXP_What do you like about the tool?", "green")
plot_checkboxes("EXP_What do you dislike about the tool?", "brown")
plot_checkboxes("EXP_This tool helps more when...", "green")

### Open Questions

In [ ]:
for col in open_cols:
    print(col)
    default_options = set(get_default_option_count(col)["Selection"].tolist()) if "comments" not in col else set()
    for row in participants[col]:
        open_response = ", ".join(list(filter(lambda x: x not in default_options, row.split(", ") if type(row) != float else [])))
        if len(open_response) == 0: continue
        print(open_response)
    print()

## Logs

## Chat Questionnaires

In [ ]:
from itertools import product
from scipy.stats import mannwhitneyu

def compare_chats(df):
    x_columns = [SHOW_COL, ORDER_COL, CAT_COL]
    y_columns = [x for x in df.columns if "how" in x.lower() and x not in x_columns]

    data = []
    for x_col, y_col in product(x_columns, y_columns):
        x_values = list(set(df[x_col].tolist()))
        x_values.sort()
        if len(x_values) != 2: continue  # control binary variables only
        a = df[df[x_col] == x_values[0]][y_col]
        b = df[df[x_col] == x_values[1]][y_col]
        s, p = mannwhitneyu(a.astype(float), b.astype(float), alternative="less")  # "less", "greater", "two-sided"
        data.append((x_col, y_col, s, p))
    chat_diff_df = pd.DataFrame(data=data, columns=["x", "y", "Mann-Whitney U statistic", "p-value"])
    chat_diff_df = chat_diff_df.sort_values(by="p-value")
    return chat_diff_df

compare_chats_df = compare_chats(chats[chats["Order"] == 1])
compare_chats_df.loc[0].to_dict()

Analyze per person's difference between chats with and without the tool

In [ ]:
df = chats
x_columns = [SHOW_COL, ORDER_COL, CAT_COL]
y_columns = [x for x in df.columns if "how" in x.lower() and x not in x_columns]
df = df.sort_values(by=["user_id", "Show Suggestions"])
df


## Correlation between Listeners and Perceptions

### Heatmap

In [ ]:
import seaborn
from seaborn import heatmap
seaborn.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white', 'figure.figsize':(11, 3)})
seaborn.diverging_palette(220, 20, as_cmap=True)

def draw_heatmap(correlations, title, x_columns):
    corr_matrix = pd.DataFrame(data={}, columns=["y",] + [x.replace(NUM_PREFIX, "") for x in x_columns])
    corr_matrix.set_index("y")
    for x, y, r, p in correlations:
        if y not in corr_matrix.index:
            corr_matrix.loc[y] = [y] + [0.0] * (len(corr_matrix.columns) - 1)
        corr_matrix.loc[y, x] = r
    
    heatmap(corr_matrix[list(filter(lambda x: x != "y", corr_matrix.columns))], yticklabels=corr_matrix["y"], xticklabels=corr_matrix.columns[1:]).set(title=title)


### Numeric Y

In [ ]:
import itertools
from scipy.stats.stats import spearmanr

def compute_num_exp_correlation(df):
    x_columns = NUM_COL_PROFILES + ["Counseling Background", "show_suggestion_first", "Category Confidence", "Participant Index"]
    y_columns = [
        'NUM_EXP_How often did the suggested strategies (blue hints)... [help?]', 
        'NUM_EXP_How often did the example responses (white buttons)... [help?]', 
        'NUM_EXP_As a whole, was the tool... [helpful?]', 
        'NUM_EXP_If the tool is provided to you, how often do you think you will make use of it? [-]', 
        'NUM_EXP_If you can switch the tool on and off as you like, will you switch it on? [-]',
    ]

    correlations = []
    for x, y in itertools.product(x_columns, y_columns):
        r, p = spearmanr(df[x], df[y])
        correlations.append((x.replace(NUM_PREFIX, ""), y.replace(NUM_PREFIX, "").replace(EXP_PREFIX, "").replace(" [-]", "").replace("...", "").replace("[", "").replace("]", ""), r, p))  # remove prefixes for readability

    draw_heatmap(correlations, "Spearman's Correlation between listeners' profile attributes and questionnaire responses converted to numerics", x_columns)

    correlations.sort(key=lambda x: x[-1])
    correlation_df = pd.DataFrame(data=correlations, columns=["Variable from Listener profile", "Variable from questionnaire response", "Spearman correlation coefficient", "Two-tailed p-value"])
    # correlation_df.to_csv("profile-questionnaire-numeric.csv")
    return correlation_df

df = compute_num_exp_correlation(participants)
df.head(5)

### Binary Y

In [ ]:
import itertools
from scipy.stats.stats import spearmanr

def compute_bin_exp_correlation(df):
    x_columns = NUM_COL_PROFILES + ["Counseling Background", "show_suggestion_first", "Category Confidence", "Participant Index"]
    y_columns = [
        'BIN_EXP_How often did the suggested strategies (blue hints)... [help?]', 
        'BIN_EXP_How often did the example responses (white buttons)... [help?]', 
        'BIN_EXP_As a whole, was the tool... [helpful?]', 
        'BIN_EXP_If the tool is provided to you, how often do you think you will make use of it? [-]', 
        'BIN_EXP_If you can switch the tool on and off as you like, will you switch it on? [-]',
    ]

    correlations = []
    for x, y in itertools.product(x_columns, y_columns):
        r, p = spearmanr(df[x], df[y])
        correlations.append((x.replace(NUM_PREFIX, ""), y.replace(BIN_PREFIX, "").replace(EXP_PREFIX, "").replace(" [-]", "").replace("...", "").replace("[", "").replace("]", ""), r, p))  # remove prefixes for readability

    draw_heatmap(correlations, "Spearman's Correlation between listeners' attributes and questionnaire responses converted to binaries", x_columns)

    correlations.sort(key=lambda x: x[-1])
    correlation_df = pd.DataFrame(data=correlations, columns=["Variable from Listener profile", "Variable from questionnaire response", "Spearman correlation coefficient", "Two-tailed p-value"])
    # correlation_df.to_csv("profile-questionnaire-binary.csv")
    return correlation_df

df = compute_bin_exp_correlation(participants)
df.head(5)

In [ ]:
copy = participants[["NUM_Listener Since", "EXP_As a whole, was the tool... [helpful?]", "OBJ_Listener Since", "Counseling Background", "Chats", "People Helped"]].copy()
copy[copy["Counseling Background"] < 0.5].sort_values("OBJ_Listener Since")